In [6]:
import os
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from time import sleep

from datetime import datetime 

In [7]:
url_base = 'https://api.yelp.com/v3/'

url_review = 'businesses/{id}/reviews'
url_business = 'businesses/search'

**get business info**

In [8]:
url = os.path.join(url_base, url_business)
api_header = {'Authorization': apikey, 'accept': 'application/json'}

for offset in range(0, 50, 50):
    params = {
        'location': 'San Francisco',
        'term': 'restaurants',
        'limit': 50,
        'offset': offset
    }

    r = requests.get(url, headers=api_header, params=params)
    
    result = json.loads(r.text)
    busi_list = result['businesses']
    busi_df = pd.DataFrame(columns=['id', 'name', 'rating', 'url', 'review_cnt'])
    for i in range(len(busi_list)):
        busi_df.loc[i, 'id'] = busi_list[i]['id']
        busi_df.loc[i, 'name'] = busi_list[i]['name']
        busi_df.loc[i, 'rating'] = busi_list[i]['rating']
        busi_df.loc[i, 'url'] = busi_list[i]['url'].split('?')[0]
        busi_df.loc[i, 'review_cnt'] = busi_list[i]['review_count']
    
    result = pd.DataFrame(columns=['id', 'name', 'rating', 'review', 'total_review'])
    idx = 0
    error_count = 0 
    for res in range(len(busi_df)):
        new_page = "?start={}"
        base_url = busi_df.loc[res, 'url']
        cnt = busi_df.loc[res, 'review_cnt']

        # soup = BeautifulSoup(content, "html.parser")
        sleep(2)
        for i in range(0, min(cnt, 100), 10):
            new_page_url = base_url + new_page.format(i)
            try: 
                new_content = requests.get(new_page_url).content
            except Exception as e:
                with open('logs.txt','w') as f:
                    timestamp = str(datetime.now())
                    error_string = f'{timestamp} : Error while using request. Error output {e} \n'
                    f.write(error_string)
                    error_count += 1

            try:

                new_soup = BeautifulSoup(new_content, "html.parser")

                relevant= new_soup.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
                for div in relevant:
                    for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
                        text = html_class.find('span')
                        review = html_class.getText()
                        result.loc[idx, 'id'] = busi_df.loc[res, 'id']
                        result.loc[idx, 'name'] = busi_df.loc[res, 'name']
                        result.loc[idx, 'rating'] = busi_df.loc[res, 'rating']
                        result.loc[idx, 'total_review'] = busi_df.loc[res, 'review_cnt']
                        result.loc[idx, 'review'] = review

                        idx += 1
            except Exception as e:
                with open('logs.txt','w') as f:
                    timestamp = str(datetime.now())
                    error_string = f'{timestamp} : Error while parsing with BeautifulSoup. Error output {e} \n'
                    f.write(error_string)
                    error_count += 1

            if error_count > 10:
                break
            
    outfile = str(offset)+'_review_result.csv'
    result.to_csv(outfile)


In [31]:
result = json.loads(r.text)
len(result['businesses'])

50

In [32]:
busi_list = result['businesses']
busi_list[0]

{'id': 'oa6ZaLdQNzZHP7--gxBh2g',
 'alias': 'california-fish-market-restaurant-san-francisco-3',
 'name': 'California Fish Market Restaurant',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/dT8Dx6y0ElNeU5Xe3funCQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/california-fish-market-restaurant-san-francisco-3?adjust_creative=bpQq5NcAeZPqAmDOGLjWgA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=bpQq5NcAeZPqAmDOGLjWgA',
 'review_count': 521,
 'categories': [{'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'fishnchips', 'title': 'Fish & Chips'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 37.7998, 'longitude': -122.40832},
 'transactions': ['pickup', 'restaurant_reservation', 'delivery'],
 'price': '$$',
 'location': {'address1': '550 Green St',
  'address2': '',
  'address3': None,
  'city': 'San Francisco',
  'zip_code': '94133',
  'country': 'US',
  'state': 'CA',
  'display_address': ['550 

In [23]:
busi_df = pd.DataFrame(columns=['id', 'name', 'rating', 'url', 'review_cnt'])
for i in range(len(busi_list)):
    busi_df.loc[i, 'id'] = busi_list[i]['id']
    busi_df.loc[i, 'name'] = busi_list[i]['name']
    busi_df.loc[i, 'rating'] = busi_list[i]['rating']
    busi_df.loc[i, 'url'] = busi_list[i]['url'].split('?')[0]
    busi_df.loc[i, 'review_cnt'] = busi_list[i]['review_count']

In [24]:
busi_df

,id,name,rating,url,review_cnt
0,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,https://www.yelp.com/biz/kothai-republic-san-f...,37
1,6YuJE0vjUpIzYckRMnzy0g,Savor,5.0,https://www.yelp.com/biz/savor-san-francisco-7,88
2,J7_-faNq_Ag9qTOlDn81Pw,Starbelly,4.0,https://www.yelp.com/biz/starbelly-san-francisco,2166
3,SUdC4LIL3huirbbBkHrhfw,The Laundromat,4.5,https://www.yelp.com/biz/the-laundromat-san-fr...,55
4,HHtpR0RslupSQ99GIIwW5A,Marufuku Ramen,4.5,https://www.yelp.com/biz/marufuku-ramen-san-fr...,4355
5,QueFVMcMlT-6aZFv2M47mg,Bottega,4.5,https://www.yelp.com/biz/bottega-san-francisco-2,541
6,f-m7-hyFzkf0HSEeQ2s-9A,Fog Harbor Fish House,4.5,https://www.yelp.com/biz/fog-harbor-fish-house...,8810
7,AtFGQccBpgEZzsy9V1330Q,Lily,4.5,https://www.yelp.com/biz/lily-san-francisco,308
8,d5752S_RDb26HxoL6O06eQ,Beit Rima,4.5,https://www.yelp.com/biz/beit-rima-san-francis...,219
9,_hOVIgjVRl_HzvLaa65KJg,The Snug,4.0,https://www.yelp.com/biz/the-snug-san-francisco,366


In [27]:
result = pd.DataFrame(columns=['id', 'name', 'rating', 'review', 'total_review'])
idx = 0
error_count = 0 
for res in range(len(busi_df)):
    new_page = "?start={}"
    base_url = busi_df.loc[res, 'url']
    cnt = busi_df.loc[res, 'review_cnt']

    # soup = BeautifulSoup(content, "html.parser")
    sleep(2)
    for i in range(0, min(cnt, 100), 10):
        new_page_url = base_url + new_page.format(i)
        try: 
            new_content = requests.get(new_page_url).content
        except Exception as e:
            with open('logs.txt','w') as f:
                timestamp = str(datetime.now())
                error_string = f'{timestamp} : Error while using request. Error output {e} \n'
                f.write(error_string)
                error_count += 1
        
        try:
            
            new_soup = BeautifulSoup(new_content, "html.parser")

            relevant= new_soup.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
            for div in relevant:
                for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
                    text = html_class.find('span')
                    review = html_class.getText()
                    result.loc[idx, 'id'] = busi_df.loc[res, 'id']
                    result.loc[idx, 'name'] = busi_df.loc[res, 'name']
                    result.loc[idx, 'rating'] = busi_df.loc[res, 'rating']
                    result.loc[idx, 'total_review'] = busi_df.loc[res, 'review_cnt']
                    result.loc[idx, 'review'] = review

                    idx += 1
        except Exception as e:
            with open('logs.txt','w') as f:
                timestamp = str(datetime.now())
                error_string = f'{timestamp} : Error while parsing with BeautifulSoup. Error output {e} \n'
                f.write(error_string)
                error_count += 1
                
        if error_count > 10:
            break
            


In [28]:
result.to_csv('50_review_result.csv')

In [29]:
result

,id,name,rating,review,total_review
0,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,"KOREAN-THAI, KOTHAI! It has a nice ring to it....",37
1,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,I heard about this Thai/Korean fusion restaura...,37
2,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,Great dishes and execution. I loved the Tom Yu...,37
3,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,Kothai Republic has so much potential and I ca...,37
4,s2R3RUJ09eBFvHVIUS4Shg,Kothai Republic,5.0,Food truck turned brick and mortar. Still defi...,37
...,...,...,...,...,...
4413,LsGkxQK217twdzZzVXsSRg,CreoLa,4.5,So excited for a new restaurant in the neighbo...,161
4414,LsGkxQK217twdzZzVXsSRg,CreoLa,4.5,We had food from here twice - Once our friends...,161
4415,LsGkxQK217twdzZzVXsSRg,CreoLa,4.5,I used to live in San Francisco and was deligh...,161
4416,LsGkxQK217twdzZzVXsSRg,CreoLa,4.5,"Their fried chicken is full of flavor, and cri...",161


In [13]:
import pandas as pd
import os

In [14]:
filelists = os.listdir('data')
# result = pd.DataFrame()
# for f in filelists:
#     tmp = pd.read_csv('data/'+f)

In [17]:
result = pd.concat([pd.read_csv('data/'+f) for f in filelists ])

In [18]:
result.to_csv('data.csv')